<a href="https://colab.research.google.com/github/Igwe-Ugo/Facial_recognition_system/blob/main/face_recog_resnet50_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [3]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.models import load_model

In [ ]:
# Load the pre-trained face embedding model
face_embedding_model = load_model('face_embedding_model.keras')

In [ ]:
# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

In [ ]:
def preprocess_face(face_img):
    # Resize the face image to 224x224 pixels (required input size for the model)
    face_img = cv2.resize(face_img, (224, 224))
    # Convert the image from BGR to RGB color space
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    # Preprocess the image using ResNet50's preprocessing function
    face_img = tf.keras.applications.resnet50.preprocess_input(face_img)
    # Add a batch dimension to the image
    return np.expand_dims(face_img, axis=0)

In [ ]:
def get_face_embedding(face_img):
    # Preprocess the face image
    preprocessed_face = preprocess_face(face_img)
    # Generate the embedding using the pre-trained model
    return face_embedding_model.predict(preprocessed_face)[0]

In [ ]:
def compare_embeddings(emb1, emb2, threshold=0.5):
    # Calculate the Euclidean distance between two embeddings
    distance = np.linalg.norm(emb1 - emb2)
    # Return True if the distance is below the threshold (faces match)
    return distance < threshold

In [ ]:
def recognize_face(frame, known_faces):
    # Convert the frame from BGR to RGB color space
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect faces in the frame using MediaPipe
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            # Get the bounding box of the detected face
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                         int(bboxC.width * iw), int(bboxC.height * ih)

            # Extract the face image from the frame
            face_img = frame[y:y+h, x:x+w]
            # Generate the embedding for the detected face
            face_embedding = get_face_embedding(face_img)

            recognized = False
            # Compare the detected face with known faces
            for name, known_embedding in known_faces.items():
                if compare_embeddings(face_embedding, known_embedding):
                    # If a match is found, label the face with the person's name
                    cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    recognized = True
                    break

            if not recognized:
                # If no match is found, label the face as "Unknown"
                cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return frame

In [ ]:
# Load known faces
known_faces = {}
known_faces_dir = "known_faces"
for person in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person)
    if os.path.isdir(person_dir):
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            # Load the image of the known person
            img = cv2.imread(img_path)
            # Generate the embedding for the known face
            face_embedding = get_face_embedding(img)
            # Store the embedding with the person's name
            known_faces[person] = face_embedding
            break  # We only need one image per person

In [ ]:
# Start video capture
cap = cv2.VideoCapture(0)

In [ ]:
while cap.isOpened():
    # Read a frame from the video capture
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame to recognize faces
    frame = recognize_face(frame, known_faces)

    # Display the processed frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()